# Util Collab

In [97]:
# !python -m pip install tensorflow keras
# !python -m pip install scikit-learn
# !python -m pip install nltk

In [98]:
# !pip install tensorflow==2.8.2

In [99]:
# !pip install --upgrade tensorflow scikeras keras

# Import

In [100]:
import numpy as np
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, matthews_corrcoef, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Flatten,BatchNormalization
from keras import regularizers

import re
import os

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\doria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Coder le texte

## Récupération des textes et des années de publications (labels)

In [120]:
# Chemin du dossier contenant les textes
dossier_textes = 'CORPUS DATATION TER'

# Récupérer la liste des noms de fichiers dans le dossier
fichiers = os.listdir(dossier_textes)

# Trier les fichiers par ordre alphabétique
fichiers.sort()

# Initialiser une liste pour stocker les textes
books = []

# Initialiser une liste pour stocker les années extraites des noms de fichiers
y = []

# Parcourir les fichiers et lire leur contenu
for fichier in fichiers[:500]:
    chemin_fichier = os.path.join(dossier_textes, fichier)
    with open(chemin_fichier, 'r', encoding='utf-8') as f:

        texte = f.read()

        # Extraire l'année du nom de fichier
        annee = re.search(r'\((\d{4})\)', fichier)  # Utilisation d'une expression régulière pour trouver l'année entre parenthèses
        # Si une année est trouvée, alors nous pouvons étudier le livre, nous l'ajoutons ainsi que son label dans les listes associées
        if annee:
            books.append(texte)
            annee_int = int(annee.group(1)) # Récupérer l'année
            y.append(annee_int)  # Ajouter l'année extraite à la liste des labels

# Division de notre bdd en deux bdd, l'une pour l'entrainement et l'autre pour les tests
X_train, X_test, y_train, y_test = train_test_split(books, y, test_size=0.1, random_state=42)
X_train,X_val,y_train,y_val =  train_test_split(X_train, y_train, test_size=0.1, random_state=42)

# Afficher les annéees extraites
print(y_train)
#print(len(X_train[9]))

[1832, 1920, 1954, 1985, 1870, 1923, 1866, 1986, 1933, 2005, 1897, 1987, 1882, 2010, 1962, 1910, 1908, 1895, 1830, 2016, 2015, 1989, 1977, 1967, 1964, 1909, 1996, 1907, 2013, 1920, 1949, 1958, 1988, 1997, 1954, 1912, 1990, 1948, 1934, 1997, 1908, 1949, 1995, 2002, 2014, 2005, 1898, 1927, 2019, 1843, 1970, 1992, 1925, 1695, 2010, 2014, 1978, 1962, 2011, 1963, 1833, 1927, 1887, 1923, 1921, 1979, 2005, 1964, 2017, 2006, 1990, 2013, 1852, 1874, 1900, 1971, 2015, 1941, 1902, 2016, 1975, 2014, 1893, 1943, 1905, 1888, 2001, 2009, 1833, 2004, 1901, 1903, 1937, 1957, 1895, 1998, 2015, 2013, 1862, 1920, 1916, 1860, 1892, 1772, 1926, 1965, 2003, 2002, 1964, 1968, 1917, 1994, 1788, 1944, 1857, 1947, 1924, 1929, 1983, 1922, 2019, 2006, 1867, 1911, 1940, 1978, 1985, 1929, 1967, 1884, 1962, 2000, 1897, 1948, 1991, 1999, 1937, 1854, 1949, 2014, 1856, 2002, 1856, 1902, 1963, 1968, 1902, 1888, 2014, 1834, 1926, 2011, 2008, 2004, 1865, 2005, 1874, 2011, 2009, 1945, 2013, 1924, 1960, 2017, 2002, 2007, 196

## Tokenization + padding

In [121]:
# Paramètres du Tokenizer
max_len = 5000 # Longueur maximale des séquences

In [122]:
class Tokenizer2:
    def __init__(self):
        self.word_to_number = {}
        self.next_number = 0

    def tokenize(self, list_of_texts):
        total_texts = len(list_of_texts)
        token_lists = []
        for i, text in enumerate(list_of_texts):
            text = text.lower()
            tokens = word_tokenize(text)
            token_lists.append([])

            for token in tokens:
                if token not in self.word_to_number:
                    self.word_to_number[token] = self.next_number
                    self.next_number += 1

                token_lists[-1].append(self.word_to_number[token])

            percentage = (i + 1) / total_texts * 100
            print(f"Progress: {percentage:.2f}% complete")

        print("\nTokenization complete.")
        return token_lists, self.next_number
    
    def tokenizeNoAdd(self, list_of_texts):
        total_texts = len(list_of_texts)
        token_lists = []
        for i, text in enumerate(list_of_texts):
            text = text.lower()
            tokens = word_tokenize(text)
            token_lists.append([])

            for token in tokens:
                if token not in self.word_to_number:
                    continue

                token_lists[-1].append(self.word_to_number[token])

            percentage = (i + 1) / total_texts * 100
            print(f"Progress: {percentage:.2f}% complete")

        print("\nTokenization complete.")
        return token_lists, self.next_number

In [123]:
def diviser_liste(liste,labels, taille):
    resultats = []
    res_labels = []
    ind = 0
    # Parcours de chaque sous-liste dans la liste principale
    for sous_liste in liste:
        longueur = len(sous_liste)
        if(longueur<taille):
            ind+=1
            continue
        nbtour = round(longueur/taille)
        lastpos=0
        # Division de la sous-liste en morceaux de taille spécifiée
        for i in range(nbtour):
            if(lastpos + taille > longueur):
              lastpos -= (lastpos + taille)-longueur

            # Ajout du morceau à la liste de résultats
            resultat = sous_liste[lastpos:lastpos + taille]
            lastpos = lastpos + taille
            resultats.append(resultat)

            # A chaque fois que je divisie, jajoute le label au meme indice
            res_labels.append(labels[ind])
        ind+=1

    return resultats,res_labels

In [124]:
tokenizer = Tokenizer2()
# Transformation de chaque texte dans X_train en une séquence d'entiers
sequences, token_number = tokenizer.tokenize(X_train)

Progress: 0.25% complete
Progress: 0.49% complete
Progress: 0.74% complete
Progress: 0.99% complete
Progress: 1.23% complete
Progress: 1.48% complete
Progress: 1.73% complete
Progress: 1.98% complete
Progress: 2.22% complete
Progress: 2.47% complete
Progress: 2.72% complete
Progress: 2.96% complete
Progress: 3.21% complete
Progress: 3.46% complete
Progress: 3.70% complete
Progress: 3.95% complete
Progress: 4.20% complete
Progress: 4.44% complete
Progress: 4.69% complete
Progress: 4.94% complete
Progress: 5.19% complete
Progress: 5.43% complete
Progress: 5.68% complete
Progress: 5.93% complete
Progress: 6.17% complete
Progress: 6.42% complete
Progress: 6.67% complete
Progress: 6.91% complete
Progress: 7.16% complete
Progress: 7.41% complete
Progress: 7.65% complete
Progress: 7.90% complete
Progress: 8.15% complete
Progress: 8.40% complete
Progress: 8.64% complete
Progress: 8.89% complete
Progress: 9.14% complete
Progress: 9.38% complete
Progress: 9.63% complete
Progress: 9.88% complete


In [125]:
data,y_train = diviser_liste(sequences,y_train,max_len)

max_words = token_number
print(y_train)
# Convertir y_train en tableau numpy
y_train = np.array(y_train)
data = np.array(data)
print(len(data))
print("Nombre d'index dans le tokenizer:", max_words)

[1832, 1832, 1832, 1832, 1832, 1832, 1832, 1832, 1832, 1832, 1832, 1832, 1832, 1832, 1832, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1920, 1954, 1954, 1954, 1954, 1954, 1954, 1954, 1954, 1954, 1954, 1954, 1954, 1954, 1954, 1954, 1954, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1985, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1870, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1923, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1866, 1986, 1986, 1986, 1986, 1986, 1986, 1986, 1986, 1986, 1986, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 1933, 193

In [126]:
len(sequences[2])

80783

# Définition des architectures des modeles testés

In [108]:
# # Construction du modèle
# # Pilee linéaire de couches. Dans ce modèle, les données passent à travers les couches dans l'ordre où elles ont été ajoutées.
# model = Sequential()
# # L'embedding est une représentation dense des mots (ou des tokens) dans un espace vectoriel.
# # L'objectif principal de l'embedding est de capturer les similarités sémantiques entre les mots en les plaçant dans un espace vectoriel où les mots ayant des significations similaires sont plus proches les uns des autres.
# model.add(Embedding(max_words, embedding_dim, input_length=max_len))

# # Couche de convolution à une dimension (1D) au modèle. Cette couche va utiliser 128 filtres de taille 10 avec une fonction d'activation ReLU (Rectified Linear Unit).
# # La convolution 1D est utilisée pour extraire des caractéristiques séquentielles des données.
# model.add(Conv1D(128,10, activation='relu'))

# # Le pooling global maximale extrait les caractéristiques les plus significatives de chaque canal de la sortie de la couche précédente, réduisant ainsi la dimensionnalité des données.
# model.add(GlobalMaxPooling1D())

# # Couche dense (entièrement connectée) au modèle avec 128 neurones et une fonction d'activation ReLU.
# model.add(Dense(128, activation='relu'))

# # Le dropout est une technique de régularisation qui désactive aléatoirement un pourcentage (dans ce cas, 50%) des neurones de la couche précédente pendant l'entraînement,
# # ce qui aide à prévenir le surapprentissage.
# model.add(Dropout(0.5))

# # Couche dense de sortie au modèle avec un seul neurone et une fonction d'activation linéaire.
# # Dans ce cas, la sortie est une seule valeur, qui est la date de publication prédite.
# model.add(Dense(1, activation='linear'))

# # La fonction ReLU est couramment utilisée dans les réseaux de neurones en raison de sa simplicité et de sa capacité à introduire une non-linéarité dans le modèle,
# # ce qui permet au réseau de capturer des motifs complexes dans les données.

# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
# model.summary()

# # Entraînement du modèle
# model.fit(data, y_train, epochs=50, batch_size=32)
# #, validation_split=0.2
# # Après l'entraînement, vous pouvez utiliser le modèle pour prédire la date de publication d'un nouveau livre.


In [127]:
embedding_dim = 50

def archi_3CONV32_F_DE_DR_DE():
  model = Sequential()

#inputdim = le nombre de voc, inputlength le nombre delement dans une liste
  model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
  model.add(Conv1D(32, 9, activation='relu'))
  model.add(Conv1D(32, 9, activation='relu'))
  model.add(Conv1D(32, 9, activation='relu'))
  model.add(Flatten())
  model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-6)))
  model.add(Dropout(0.5))
  model.add(Dense(1))

  return model

def archi_3CONV16_F_DE_DR_DE():
  model = Sequential()

  model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
  model.add(Conv1D(16, 5, activation='relu'))
  model.add(Conv1D(16, 5, activation='relu'))
  model.add(Conv1D(16, 5, activation='relu'))
  model.add(Flatten())
  model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-6)))
  model.add(Dropout(0.5))
  model.add(Dense(1))
  return model

def archi_3CONV64_F_DE_DR_DE():
  model = Sequential()

  model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
  model.add(Conv1D(64, 9, activation='relu'))
  model.add(Conv1D(64, 9, activation='relu'))
  model.add(Conv1D(64, 9, activation='relu'))
  model.add(Flatten())
  model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-6)))
  model.add(Dropout(0.5))
  model.add(Dense(1))

  return model

def archi_3CONV64_32_16_F_DE_DR_DE():
  model = Sequential()

  model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len))
  model.add(Conv1D(64, 5, activation='relu'))
  model.add(Conv1D(32, 5, activation='relu'))
  model.add(Conv1D(16, 5, activation='relu'))
  model.add(Flatten())
  model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-6)))
  model.add(Dropout(0.5))
  model.add(Dense(1))

  return model

# Prétraitement données de test

In [128]:
# Transformation de chaque texte dans X_train en une séquence d'entiers
sequences_test, _ = tokenizer.tokenizeNoAdd(X_test)

data_test,y_test = diviser_liste(sequences_test,y_test,max_len)

# Convertir y_train en tableau numpy
y_test = np.array(y_test)
data_test = np.array(data_test)

# Transformation de chaque texte dans X_train en une séquence d'entiers
sequences_val, _ = tokenizer.tokenizeNoAdd(X_val)

data_val,y_val = diviser_liste(sequences_val,y_val,max_len)

# Convertir y_train en tableau numpy
y_val = np.array(y_val)
data_val = np.array(data_val)

Progress: 2.00% complete
Progress: 4.00% complete
Progress: 6.00% complete
Progress: 8.00% complete
Progress: 10.00% complete
Progress: 12.00% complete
Progress: 14.00% complete
Progress: 16.00% complete
Progress: 18.00% complete
Progress: 20.00% complete
Progress: 22.00% complete
Progress: 24.00% complete
Progress: 26.00% complete
Progress: 28.00% complete
Progress: 30.00% complete
Progress: 32.00% complete
Progress: 34.00% complete
Progress: 36.00% complete
Progress: 38.00% complete
Progress: 40.00% complete
Progress: 42.00% complete
Progress: 44.00% complete
Progress: 46.00% complete
Progress: 48.00% complete
Progress: 50.00% complete
Progress: 52.00% complete
Progress: 54.00% complete
Progress: 56.00% complete
Progress: 58.00% complete
Progress: 60.00% complete
Progress: 62.00% complete
Progress: 64.00% complete
Progress: 66.00% complete
Progress: 68.00% complete
Progress: 70.00% complete
Progress: 72.00% complete
Progress: 74.00% complete
Progress: 76.00% complete
Progress: 78.00%

# Entrainement des models

## Trouver le meilleur

In [111]:
# # Définir la liste des modèles à évaluer
# models = [archi_3CONV32_F_DE_DR_DE(), archi_3CONV16_F_DE_DR_DE(), archi_3CONV64_F_DE_DR_DE(), archi_3CONV64_32_16_F_DE_DR_DE()]

# best_model = None
# best_val_loss = float('inf')
# best_val_mae = float('inf')

# # Entraîner et évaluer chaque modèle
# for model in models:
#     model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
#     model.fit(data, y_train, epochs=20, batch_size=64, validation_data=(data_val, y_val))

#     # Évaluer sur l'ensemble de test
#     val_loss, val_mae = model.evaluate(data_test, y_test)

#     # Mettre à jour le meilleur modèle si nécessaire
#     if val_mae < best_val_mae:
#         best_val_loss = val_loss
#         best_val_mae = val_mae
#         best_model = model

# # Afficher les performances du meilleur modèle
# print("Le meilleur modèle est:")
# best_model.summary()
# print("Meilleure valeur de perte (loss) sur l'ensemble de test:", best_val_loss)
# print("Meilleure MAE (Mean Absolute Error) sur l'ensemble de test:", best_val_mae)


In [112]:
# model = best_model

## Ou en entrainer un seul

In [133]:
model = archi_3CONV64_F_DE_DR_DE()

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
model.summary()

from keras.callbacks import EarlyStopping

# Création du callback EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=1)  # Vous pouvez ajuster la patience selon vos besoins

# Entraînement du modèle avec EarlyStopping
history = model.fit(data, y_train, epochs=20, batch_size=64, validation_data=(data_val, y_val), callbacks=[early_stopping])

# L'historique de l'entraînement sera disponible dans history.history comme précédemment

history.history

Model: "sequential_26"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_26 (Embedding)    (None, 5000, 50)          9047850   
                                                                 
 conv1d_78 (Conv1D)          (None, 4992, 64)          28864     
                                                                 
 conv1d_79 (Conv1D)          (None, 4984, 64)          36928     
                                                                 
 conv1d_80 (Conv1D)          (None, 4976, 64)          36928     
                                                                 
 flatten_26 (Flatten)        (None, 318464)            0         
                                                                 
 dense_52 (Dense)            (None, 256)               81527040  
                                                                 
 dropout_26 (Dropout)        (None, 256)             

{'loss': [288177.34375, 38918.734375],
 'mae': [340.5596923828125, 157.28672790527344],
 'val_loss': [3040.67919921875, 4944.1337890625],
 'val_mae': [49.08445739746094, 59.665740966796875]}

In [130]:
# Evaluate the model on the test data
val_loss, val_mae = model.evaluate(data_test, y_test)

# Print the validation loss and validation mean absolute error
print("Validation Loss:", val_loss)
print("Validation MAE:", val_mae)

23/23 [==============================] - 3s 108ms/step - loss: 4258.8364 - mae: 59.6753
Validation Loss: 4258.83642578125
Validation MAE: 59.67530059814453
